In [105]:
from functools import partial, singledispatch, wraps
from sympy import sympify
from sympy import integrate as _integrate
from mpmath import e
import logging
import sympy

sympy.init_printing()

def coerce(T):
    """
    Coerce the output of the wrapped function to the type
    specified by T.
    """
    def outer(f):
        @wraps(f)
        def inner(*args, **kwargs):
            result = f(*args, **kwargs)
            try:
                return T(result)
            except Exception as e:
                logging.exception(f'could not coerce {result} to type {T}')
                return result
        return inner
    return outer

def _sum(left_or_right, y_values, delta_x):
    """
    Return the left or right hand sum of a list of y values.
    """
    # make sure y_values is a list
    y_values = list(y_values)
    
    if left_or_right == 'left':
        y_values = y_values[:-1]
    elif left_or_right == 'right':
        y_values = y_values[1:]
        
    return sum(y*delta_x for y in y_values)

lhs = partial(_sum, 'left')
rhs = partial(_sum, 'right')

@singledispatch
def integrate():
    pass

@integrate.register(dict)        
def _(observations, a_b=None, n=None):
    """
    Given a table of x, y values as a dictionary
    return the definite integral.
    """
    x_vals = list(observations.keys())
    delta_x = x_vals[1]-x_vals[0]
    
    # n is the number of times we evenly split the observations
    # by default, we'll choose the maximum amount of splits possible
    
    n = len(observations)-1 if n is None else n
    
    # let a and b be the largest and smallest values
    # in our list of x values by default
    
    if a_b is not None:
        a, b = a_b
    else:
        a = x_vals[0]
        b = x_vals[-1]
    
    # create the list of y values that we will compute
    # the right and left hand sums of
    start = x_vals.index(a)
    stop = x_vals.index(b)+1
    step = len(observations) // n    
    y_vals = list(observations.values())[start:stop:step]
    
    # return the average of the left and right hand sums
    return (lhs(y_vals, delta_x) + rhs(y_vals, delta_x)) / 2

@integrate.register(sympy.Basic)
@coerce(float)
def _(expr, a_b=None):
    """
    Return the integral of a single-variable expression.
    """
    # ensure we replace e variable with constant e in
    # expression
    expr = expr.subs('e', e)
    
    
    # extract symbol
    if a_b is not None:
        a, b = a_b
        var = list(expr.free_symbols)[0]
        return _integrate(expr, (var, a, b))
    
    return _integrate(expr)

@integrate.register(str)
def _(expr, a_b=None):
    """
    Return the integral of a single-variable expression.
    """
    return integrate(sympify(expr), a_b=a_b)
    
    

In [106]:
velocities = [5, 7, 11, 18, 29, 45]

lhs(velocities, 2) # -> 140
rhs(velocities, 2) # -> 220

observations = dict(zip(range(20, 31, 2), velocities))
observations # -> {20: 5, 22: 7, 24: 11, 26: 18, 28: 29, 30: 45}

integrate(observations) # -> 180 